In [15]:
from torch_geometric.data import Data
import torch
import torch.nn as nn
import torch.nn.functional as F 
import random
from torch_geometric.datasets import CoraFull, TUDataset
import numpy as np
import torch_geometric as tg
from utils import count_in_degree, count_out_degree, create_pointer_graph
from torch.utils.tensorboard import SummaryWriter

In [9]:
# graphs = TUDataset("", "ENZYMES")
graphs = TUDataset("", "PROTEINS")
cora = CoraFull("", "Cora").data
perm = np.random.choice(list(range(len(graphs))), len(graphs), replace=False)
graphs = [graphs[int(i)] for i in perm]
ys = [graph.y.unsqueeze(0) for graph in graphs]
ys = torch.cat(ys)

In [34]:
graphs[mini].edge_index

tensor([[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7,
         7, 7, 7, 8, 8, 8, 8, 9, 9, 9],
        [2, 3, 4, 7, 8, 9, 0, 3, 4, 0, 2, 4, 5, 0, 2, 3, 5, 3, 4, 6, 5, 7, 8, 1,
         6, 8, 9, 1, 6, 7, 9, 1, 7, 8]])

In [33]:
mini, minv = -1, 100000
maxv = 10
for i, graph in enumerate(graphs):
    print(graph.num_nodes)
    if minv > graph.num_nodes and maxv <= graph.num_nodes:
        minv = graph.num_nodes
        mini = i
minv, mini

13
10
18
5
10
146
22
14
7
96
66
38
46
32
26
36
34
26
16
14
13
16
21
54
186
22
17
6
15
27
41
24
84
21
23
38
22
11
39
93
15
125
40
108
34
22
105
11
22
8
17
40
94
5
82
153
50
20
17
11
23
11
4
30
54
40
52
20
15
36
156
24
50
33
15
10
15
38
96
15
39
19
21
26
21
31
58
20
43
23
77
46
14
116
24
27
22
31
14
10
17
28
6
17
151
6
27
35
122
41
45
15
33
10
10
96
16
23
36
8
14
89
25
30
27
14
36
36
56
19
88
13
12
31
11
44
21
17
48
33
30
24
16
58
10
246
59
42
41
57
24
23
12
16
24
32
10
20
24
32
23
21
189
59
94
5
11
24
27
14
33
10
39
23
20
7
32
52
8
62
19
80
9
16
30
12
20
28
38
34
159
30
14
34
10
33
18
19
15
55
15
27
24
81
35
24
7
6
31
9
30
109
48
20
111
29
18
50
80
22
12
12
88
28
57
39
136
6
20
4
15
39
4
7
12
8
34
48
15
6
94
21
47
21
73
50
59
5
41
13
49
20
52
25
30
9
32
226
33
113
28
74
30
39
23
23
64
18
17
38
244
26
14
20
5
18
54
6
95
12
27
38
84
123
23
34
43
149
47
18
52
12
24
39
20
35
87
4
18
28
19
8
30
5
21
504
60
21
24
20
108
33
48
23
12
87
38
18
8
20
33
22
26
13
64
30
8
54
38
43
10
23
8
32
33
20
8

(10, 1)

In [47]:
numel = 10
ei = torch.tensor([list(range(numel)), [(i+1)%numel for i in range(numel)]])
ei2= torch.cat((ei[1].unsqueeze(0), ei[0].unsqueeze(0)))
ei = torch.cat((ei, ei2), dim=1)
xi = torch.zeros(numel, 1)
graphi = Data(x=xi, edge_index=ei)
create_pointer_graph(graphi), ei

([([1, 9], [9, 1]),
  ([2, 0], [0, 2]),
  ([3, 1], [1, 3]),
  ([4, 2], [2, 4]),
  ([5, 3], [3, 5]),
  ([6, 4], [4, 6]),
  ([7, 5], [5, 7]),
  ([8, 6], [6, 8]),
  ([9, 7], [7, 9]),
  ([0, 8], [8, 0])],
 tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0],
         [1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]))

In [58]:
def bfs(graph, node):
    vals = torch.zeros(graph.x.shape[0], 1)-1
    pg = create_pointer_graph(graph)
    expanded, toexpand, newset = set(), [node], []
    vals[node, 0] = 0
    step = 1
    lastlen = len(expanded)+1
    while lastlen != len(expanded):
        lastlen = len(expanded)
        for toex in toexpand:
            for candidate in pg[toex][0]:
                if candidate not in expanded:
                    vals[candidate] = vals[candidate]*0 + step
                    newset.append(candidate)
            expanded.add(toex)
        toexpand = newset
        newset = []
        step += 1
        # print(expanded, toexpand, newset, lastlen)
    return vals
def get_diamenter(graph):
    max_dist = -1
    for node in range(graph.num_nodes):
        vals = bfs(graph, node)
        max = torch.max(vals)
        if max_dist < max:
            max_dist = max
    return max_dist

bfs(graphs[mini], 8), 
get_diamenter(graphs[10])
# bfs(graphi, 0)

tensor(34.)

In [59]:
degs = []
for graph in graphs:
    degs.append({"in":count_in_degree(graph), "out":count_out_degree(graph)})

In [60]:
embdim = 16
for graph in graphs:
    zeros = torch.zeros(graph.x.shape[0], 16-graph.x.shape[1])
    graph.x = torch.cat((graph.x, zeros), dim=1)

In [61]:
maxi = -1
for graph in graphs:
    maxi = torch.max(torch.max(graph.y), torch.zeros_like(graph.y)+maxi)
print(maxi)

tensor([1])


In [62]:
class GCNLayer(nn.Module):
    def __init__(self, embdim):
        super().__init__()
        self.lin = nn.Linear(embdim, embdim)
        self.mlp = MLP([embdim, 16, embdim])
    def forward(self, x, graph, igraph):
        newx = torch.zeros_like(x)

        #get features
        # h = self.lin(x)
        h = x

        #add self loops
        tmp = torch.range(0, graph.num_nodes-1).unsqueeze(0)
        tmp = torch.cat((tmp, tmp)).long()
        graph.edge_index = torch.cat((graph.edge_index, tmp), dim=1)

        #pass messages
        for outnode, innode in zip(graph.edge_index[0], graph.edge_index[1]):
            newx[innode] += h[outnode]*1/((1+degs[igraph]["in"][innode])*(1+degs[igraph]["out"][outnode]))

        newx = self.mlp(newx)
        return newx

class MLP(nn.Module):
    def __init__(self, arch, last_activation=F.relu):
        super().__init__()
        self.lins = nn.ModuleList([nn.Linear(a, b) for a, b in zip(arch[:-1], arch[1:])])
        self.last_activation = last_activation
    def forward(self, x):
        h = x
        for lin in self.lins[:-1]:
            h = F.relu(lin(h)) #+h
        h = self.last_activation(self.lins[-1](h))
        return h

class Model(nn.Module):
    def __init__(self, n, embdim=16):
        super().__init__()
        self.layers = nn.ModuleList([GCNLayer(embdim) for i in range(n)])
        self.classifier = MLP( [embdim, 16, 16, 2], last_activation= lambda x: torch.softmax(x, dim=-1))
        self.embdim = embdim
    def forward(self, graph, igraph):
        h = graph.x.clone()
        zeros = torch.zeros(h.shape[0], self.embdim-h.shape[1])
        h = torch.cat((h, zeros), dim=1)     

        for layer in self.layers:
            h = layer(h, graph, igraph)
        x = h.mean(dim=0, keepdim=True)
        preds = self.classifier(x)
        return preds

model = Model(100)

In [63]:
opt = torch.optim.Adam(model.parameters(), lr=0.001)

In [42]:
! mkdir tb
writer = SummaryWriter('tb/non_test_huge100') 

bsize, bpass, ibatch = 8, 0, 0
bpreds = torch.zeros(0, 2)
# ys = graphs.data.y
for ep in range(10):
    for igraph, graph in enumerate(graphs[:]):
        preds = model(graph, igraph)
        if bpass < bsize:
            bpreds = torch.cat((bpreds, preds))
            bpass += 1
        else:
            y = ys[ibatch*bsize:(ibatch+1)*bsize]
            yhot = F.one_hot(y.squeeze(-1), num_classes=2)
            # print(y.shape, bpreds.shape)
            # print(bpreds)
            # print(yhot)
            # print(y)
            # loss = F.cross_entropy(bpreds, yhot)
            loss = -(yhot*torch.log(bpreds+1e-8) + (1-yhot)*torch.log(1-bpreds+1e-8)).mean()

            acc = ((bpreds>0.5).squeeze(-1)==y).float().mean()
            opt.zero_grad()
            loss.backward()
            opt.step()
            ibatch += 1
            bpass = 0
            print(bpreds)
            bpreds = torch.zeros(0, 2)
            writer.add_scalar("loss", loss.item(), ibatch)
            writer.add_scalar("acc" , acc.item() , ibatch)
            # print(torch.argmax(preds, dim=1))
            # print(y)
            print(ep, ibatch, loss.item(), acc.item())



mkdir: cannot create directory ‘tb’: File exists
tensor([[0.5280, 0.4720],
        [0.6125, 0.3875],
        [0.5282, 0.4718],
        [0.5313, 0.4687],
        [0.5308, 0.4692],
        [0.5317, 0.4683],
        [0.5317, 0.4683],
        [0.5279, 0.4721]], grad_fn=&lt;CatBackward&gt;)
0 1 0.6638650894165039 0.5
tensor([[0.5325, 0.4675],
        [0.5325, 0.4675],
        [0.5325, 0.4675],
        [0.5325, 0.4675],
        [0.5325, 0.4675],
        [0.5325, 0.4675],
        [0.5325, 0.4675],
        [0.5325, 0.4675]], grad_fn=&lt;CatBackward&gt;)
0 2 0.662712812423706 0.5
tensor([[0.5341, 0.4659],
        [0.5341, 0.4659],
        [0.5341, 0.4659],
        [0.5342, 0.4658],
        [0.5341, 0.4659],
        [0.5341, 0.4659],
        [0.5341, 0.4659],
        [0.5341, 0.4659]], grad_fn=&lt;CatBackward&gt;)
0 3 0.6612917184829712 0.5
tensor([[0.5358, 0.4642],
        [0.5358, 0.4642],
        [0.5358, 0.4642],
        [0.5358, 0.4642],
        [0.5358, 0.4642],
        [0.5359, 0.4641],
 

KeyboardInterrupt: 

In [104]:
preds = torch.randn_like(yhot.float())
print(torch.argmax(preds, dim=1).shape, y.shape, yhot.shape, preds.shape)
(torch.argmax(preds, dim=1)==y).float().mean()

torch.Size([4]) torch.Size([4]) torch.Size([4, 70]) torch.Size([4, 70])


tensor(0.)

In [23]:
graphs.data

Data(edge_index=[2, 162088], x=[43471, 3], y=[1113])